In [3]:
import pandas as pd


df = pd.read_json('./test.jsonl', lines=True)
df = df[['sentence', 'intent', 'entities']]
df = df.drop(index=0)
df.reset_index(drop=True)
df.to_json('./test.json', orient='records')

df = pd.read_json('./test.json')
df

,sentence,intent,entities
0,put meeting with pawel for tomorrow ten am,calendar_set,"[{'span': [1], 'type': 'event_name'}, {'span':..."
1,what is the exchange rate of us dollar to poun...,news_query,"[{'span': [3, 4, 5, 6, 7, 8, 9, 10], 'type': '..."
2,increase the brightness of the lights,hue_lightup,[]
3,which film the music comes from what is the na...,music_query,[]
4,turn up the brightness,iot_hue_lightup,[]
...,...,...,...
2968,video games,play_game,[]
2969,play the weekend,play_music,"[{'span': [2], 'type': 'song_name'}]"
2970,submit a negative review about a company,social_post,[]
2971,i need you to get me up at six am,alarm_set,"[{'span': [8, 9], 'type': 'time'}]"


In [21]:
import pandas as pd
import json

with open('./train.json') as file:
    data = json.load(file)
    df = pd.DataFrame(data)
    types = pd.json_normalize(data, record_path='entities')
    _, type_labels = pd.factorize(types['type'])
    type_labels = type_labels.to_numpy().tolist()
    df['intent'], intent_labels = pd.factorize(df['intent'])
    intent_labels = intent_labels.to_numpy().tolist()
    
    type_labels.append('O')
    
    pd.DataFrame(intent_labels).to_csv('./intent_labels.csv', index=False)
    pd.DataFrame(type_labels).to_csv('./type_labels.csv', index=False)
    
    

In [13]:
from transformers import AutoTokenizer, BertModel, BertConfig
import pandas as pd

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

df = pd.read_json('./test_t.json')
tz = tokenizer(df['sentence'].to_numpy().tolist())

invalid_idx = []

for idx in range(len(df)):
    if len(df['sentence'][idx].split()) <= max([v for v in tz.word_ids(idx) if v != None]):
        invalid_idx.append(idx)

df = df.drop(index=invalid_idx)
df = df.reset_index(drop=True)


In [14]:
type_labels = pd.read_csv('./type_labels.csv').iloc[:,0].to_numpy().tolist()

def entity_to_list(entry):
    
    
    
    sentence_len = len(entry['sentence'].split())
    entities = entry['entities']
    
    
    ret = [type_labels.index('O')] * sentence_len
    
    for idx in range(sentence_len):
        for entity in entities:
            if idx in entity['span']:
                ret[idx] = type_labels.index(entity['type'])
                

        
    return ret
        

df['entities'] = df.apply(entity_to_list, axis=1)

#df.iloc[0:-1000].to_json('./train.json', orient='records')
#df.iloc[-1000:].to_json('./valid.json', orient='records')
df.to_json('./test.json', orient='records')



In [4]:
import pandas as pd

df = pd.read_json('./test.json')
df.iloc[0:1000].to_json('./valid.json', orient='records')

In [19]:
df = pd.read_json('./test.json')
df

,sentence,intent,entities
0,put meeting with pawel for tomorrow ten am,calendar_set,"[55, 5, 55, 1, 55, 0, 2, 2]"
1,what is the exchange rate of us dollar to poun...,news_query,"[55, 55, 55, 13, 13, 13, 13, 13, 13, 13, 13]"
2,increase the brightness of the lights,hue_lightup,"[55, 55, 55, 55, 55, 55]"
3,which film the music comes from what is the na...,music_query,"[55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 5..."
4,turn up the brightness,iot_hue_lightup,"[55, 55, 55, 55]"
...,...,...,...
2711,video games,play_game,"[55, 55]"
2712,play the weekend,play_music,"[55, 55, 42]"
2713,submit a negative review about a company,social_post,"[55, 55, 55, 55, 55, 55, 55]"
2714,i need you to get me up at six am,alarm_set,"[55, 55, 55, 55, 55, 55, 55, 55, 2, 2]"


In [3]:
import pandas as pd

df = pd.read_csv('type_labels.csv')
df

,0
0,date
1,person
2,time
3,list_name
4,business_name
5,event_name
6,relation
7,playlist_name
8,definition_word
9,email_address
